In [1]:
# Import my Quantum Circuit Simulator
from QCirc import QuantumCircuit
from QGSTates import QuantumGate, QuantumState

In [2]:
qc = QuantumCircuit(2)
qc.add_gate(QuantumGate.H(), 0)
qc.add_gate(QuantumGate.CNOT(), [0, 1])

final_state = qc.run()
print(final_state.vector)

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
qc = QuantumCircuit(20)
qc.add_gate(QuantumGate.H(), 0)
qc.add_gate(QuantumGate.CNOT(), [0, 1])

final_state = qc.run()
print(final_state.vector)

[0.70710678+0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
 0.        +0.j 0.        +0.j]


In [4]:
def quantum_fourier_transform(qc):
    """
    Applies the Quantum Fourier Transform to the given QuantumCircuit object.
    The circuit must already be initialized with the desired number of qubits.
    """
    n = qc.num_qubits
    for j in range(n):
        qc.add_gate(QuantumGate.H(), j)
        for k in range(j+1, n):
            angle = 2 * 3.141592653589793 / (2 ** (k - j + 1))
            qc.add_gate(QuantumGate.CRZ(angle), [k, j])
    # Optionally, swap qubits to reverse order
    for i in range(n // 2):
        qc.add_gate(QuantumGate.SWAP(), [i, n - i - 1])

qc = QuantumCircuit(20)
quantum_fourier_transform(qc)
print(qc.number_of_gates)
final_state = qc.run()
print(final_state.vector)



220
[0.00097656+0.j 0.00097656+0.j 0.00097656+0.j ... 0.00097656+0.j
 0.00097656+0.j 0.00097656+0.j]
[0.00097656+0.j 0.00097656+0.j 0.00097656+0.j ... 0.00097656+0.j
 0.00097656+0.j 0.00097656+0.j]


In [5]:
qft_qc = QuantumCircuit(20)
# start from |0>^20 but lets start from |1>^20
for i in range(20):
    qft_qc.add_gate(QuantumGate.H(), i)
quantum_fourier_transform(qft_qc)
print(qft_qc.number_of_gates)
fs = qft_qc.run()
print(fs.vector)

240
[-2.47470220e-09-2.55085235e-03j  0.00000000e+00+0.00000000e+00j
  1.50215785e-08+2.13471703e-03j ...  0.00000000e+00+0.00000000e+00j
  5.39240045e-10-9.92041146e-05j  0.00000000e+00+0.00000000e+00j]
[-2.47470220e-09-2.55085235e-03j  0.00000000e+00+0.00000000e+00j
  1.50215785e-08+2.13471703e-03j ...  0.00000000e+00+0.00000000e+00j
  5.39240045e-10-9.92041146e-05j  0.00000000e+00+0.00000000e+00j]


In [10]:
# Apply QFT 1, 2, 3, and 4 times starting from |0>^20 and print the final state each time
for times in range(1, 5):
    qc = QuantumCircuit(20)
    print(qc.run().vector)
    for _ in range(times):
        quantum_fourier_transform(qc)
    state = qc.run()
    print(f"After applying QFT {times} time(s):")
    print(state.vector)

[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
After applying QFT 1 time(s):
[0.00097656+0.j 0.00097656+0.j 0.00097656+0.j ... 0.00097656+0.j
 0.00097656+0.j 0.00097656+0.j]
[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
After applying QFT 2 time(s):
[-2.47470220e-09-2.55085235e-03j  0.00000000e+00+0.00000000e+00j
  1.50215785e-08+2.13471703e-03j ...  0.00000000e+00+0.00000000e+00j
  5.39240045e-10-9.92041146e-05j  0.00000000e+00+0.00000000e+00j]
[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
After applying QFT 3 time(s):
[ 0.01232048-0.01305346j  0.00111091-0.00251707j  0.00251707+0.0011109j
 ... -0.00909493+0.00695465j -0.00695468-0.00909491j
 -0.01305342-0.01232052j]
[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
After applying QFT 4 time(s):
[-6.82861022e-02+1.67227455e-01j  1.94783288e-19+3.00559261e-19j
  5.53790177e-02+1.56934205e-02j ... -1.28999040e-19+1.01051980e-19j
 -2.09822929e-02+1.63163154e-03j -2.68162126e-19+4.65216614e-20j]


In [6]:
import numpy as np

def theoretical_qft_state(n, basis_index):
    """Returns the theoretical QFT of |basis_index> for n qubits."""
    N = 2 ** n
    state = np.zeros(N, dtype=complex)
    for k in range(N):
        state[k] = np.exp(2j * np.pi * basis_index * k / N)
    return state / np.sqrt(N)

def reverse_bits(x, n):
    result = 0
    for i in range(n):
        if (x >> i) & 1:
            result |= 1 << (n - 1 - i)
    return result

for n in range(2, 6):
    N = 2 ** n
    for basis_index in range(N):
        # Prepare |basis_index> state
        qc = QuantumCircuit(n)
        for q in range(n):
            if (basis_index >> q) & 1:
                qc.add_gate(QuantumGate.X(), q)
                # qc.add_gate(QuantumGate.X(), q) if q%2 == 0 else qc.add_gate(QuantumGate.H(), q)
        # Apply QFT
        quantum_fourier_transform(qc)
        sim_state = qc.run().vector
        # Reverse bits for theoretical state
        theor_state = theoretical_qft_state(n, reverse_bits(basis_index, n))
        # Compare
        fidelity = np.abs(np.vdot(sim_state, theor_state)) ** 2
        print(f"n={n}, basis_index={basis_index}, fidelity={fidelity:.6f}")
        assert np.allclose(np.abs(fidelity), 1.0, atol=1e-6), f"QFT failed for n={n}, basis={basis_index}"

n=2, basis_index=0, fidelity=1.000000
n=2, basis_index=1, fidelity=1.000000
n=2, basis_index=2, fidelity=1.000000
n=2, basis_index=3, fidelity=1.000000
n=3, basis_index=0, fidelity=1.000000
n=3, basis_index=1, fidelity=1.000000
n=3, basis_index=2, fidelity=1.000000
n=3, basis_index=3, fidelity=1.000000
n=3, basis_index=4, fidelity=1.000000
n=3, basis_index=5, fidelity=1.000000
n=3, basis_index=6, fidelity=1.000000
n=3, basis_index=7, fidelity=1.000000
n=4, basis_index=0, fidelity=1.000000
n=4, basis_index=1, fidelity=1.000000
n=4, basis_index=2, fidelity=1.000000
n=4, basis_index=3, fidelity=1.000000
n=4, basis_index=4, fidelity=1.000000
n=4, basis_index=5, fidelity=1.000000
n=4, basis_index=6, fidelity=1.000000
n=4, basis_index=7, fidelity=1.000000
n=4, basis_index=8, fidelity=1.000000
n=4, basis_index=9, fidelity=1.000000
n=4, basis_index=10, fidelity=1.000000
n=4, basis_index=11, fidelity=1.000000
n=4, basis_index=12, fidelity=1.000000
n=4, basis_index=13, fidelity=1.000000
n=4, bas

In [9]:
# Test QFT periodicity: applying QFT 4 times should return the original state (up to global phase)

def equal_up_to_global_phase(a, b, tol=1e-6):
    '''Return True if vectors a and b are equal up to a global phase.'''
    a = np.asarray(a)
    b = np.asarray(b)
    if np.allclose(a, 0) or np.allclose(b, 0):
        return np.allclose(a, b, atol=tol)
    # Remove global phase by normalizing first nonzero entry
    def normalize_global_phase(vec):
        idx = np.flatnonzero(np.abs(vec) > tol)
        if len(idx) == 0:
            return vec
        phase = vec[idx[0]] / abs(vec[idx[0]])
        return vec / phase
    a_norm = normalize_global_phase(a)
    b_norm = normalize_global_phase(b)
    return np.allclose(a_norm, b_norm, atol=tol)

n = 5
for basis_index in range(2 ** n):
    qc = QuantumCircuit(n)
    for q in range(n):
        if (basis_index >> q) & 1:
            qc.add_gate(QuantumGate.X(), q)
    original_state = qc.run().vector.copy()
    for _ in range(4):
        quantum_fourier_transform(qc)
    final_state = qc.run().vector
    # Check if states are equal up to global phase
    print(f"basis_index={basis_index}, equal_up_to_global_phase={equal_up_to_global_phase(original_state, final_state)}")
    assert equal_up_to_global_phase(original_state, final_state), f"QFT periodicity failed for basis_index={basis_index}"

basis_index=0, equal_up_to_global_phase=False


AssertionError: QFT periodicity failed for basis_index=0